In [1]:
!pip install --quiet transformers evaluate datasets baal

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 67.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install --quiet --upgrade accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 16.5 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!cp "/content/drive/MyDrive/RuSportSum/train_selector_during_match2.csv" /content/train.csv
!cp "/content/drive/MyDrive/RuSportSum/eval_selector_during_match2.csv" /content/eval.csv
!cp -r "/content/drive/MyDrive/RuSportSum/bald" "/content/bald"

In [5]:
import os
import random
from copy import deepcopy

import pandas as pd
import numpy as np
import torch.backends
import transformers
import evaluate

from datasets import load_dataset
from tqdm import tqdm
from transformers import BertForSequenceClassification
from transformers import BertTokenizer, TrainingArguments
from transformers import set_seed

from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    PretrainedConfig,
    set_seed,
    Trainer,
    TrainingArguments
)

transformers.utils.logging.set_verbosity_warning()

from baal.active import get_heuristic
from baal.active.dataset.nlp_datasets import (
    active_huggingface_dataset,
    HuggingFaceDatasets,
)
from baal.bayesian.dropout import patch_module
from baal.transformers_trainer_wrapper import BaalTransformersTrainer
from baal.active import get_heuristic, ActiveLearningDataset
from baal.active.dataset.base import Dataset

from typing import List
from pprint import pprint

SEED = 3

random.seed(SEED)
torch.manual_seed(SEED)

# Set tranformer seed to ensure that initial weights are identical
set_seed(SEED)

In [6]:
train_data = pd.read_csv("/content/train.csv")
eval_data = pd.read_csv("/content/eval.csv")

In [7]:
train_data

,match_id,news_id,name,team1,team2,relevant_part,type,minute,content,news_name_body
0,787156,158820706,Россия. Премьер-лига,Ростов,Амкар,0,empty,0,0 минута. Приветствуем Всех любителей самой по...,«Ростов» – «Амкар». Вавжиняк удален с поля & З...
1,787156,158820706,Россия. Премьер-лига,Ростов,Амкар,1,empty,0,0 минута. Предлагаем вашему вниманию прямую те...,«Ростов» – «Амкар». Вавжиняк удален с поля & З...
2,787156,158820706,Россия. Премьер-лига,Ростов,Амкар,0,empty,0,0 минута. Ростовчане весьма уверенно выступают...,«Ростов» – «Амкар». Вавжиняк удален с поля & З...
3,787156,158820706,Россия. Премьер-лига,Ростов,Амкар,0,empty,0,"0 минута. Более того, будь ""Ростов"" чуть удачл...",«Ростов» – «Амкар». Вавжиняк удален с поля & З...
4,787156,158820706,Россия. Премьер-лига,Ростов,Амкар,0,empty,0,"0 минута. ""Амкар"", подумать только, всего на 3...",«Ростов» – «Амкар». Вавжиняк удален с поля & З...
...,...,...,...,...,...,...,...,...,...,...
328467,1013052,1027838659,Англия. Кубок,Манчестер Юнайтед,Арсенал,0,empty,95,95 минута. Сейчас пойдет компенсированное врем...,«МЮ» – «Арсенал». Окслейд-Чемберлен получил тр...
328468,1013052,1027838659,Англия. Кубок,Манчестер Юнайтед,Арсенал,0,attention,96,"96 минута. Все! 1:2 - впервые за 9 лет ""Арсена...",«МЮ» – «Арсенал». Окслейд-Чемберлен получил тр...
328469,1013052,1027838659,Англия. Кубок,Манчестер Юнайтед,Арсенал,0,empty,96,"96 минута. Руни снова забил ""канонирам"", но ег...",«МЮ» – «Арсенал». Окслейд-Чемберлен получил тр...
328470,1013052,1027838659,Англия. Кубок,Манчестер Юнайтед,Арсенал,0,empty,97,"97 минута. Спасибо, что были с нами! Завтра от...",«МЮ» – «Арсенал». Окслейд-Чемберлен получил тр...


In [8]:
data_for_predict = train_data.loc[21_015:]
data_for_predict.to_csv('data_for_predict.csv', index=False, encoding='utf-8')

In [9]:
load_data = load_dataset("csv", data_files={'train': '/content/data_for_predict.csv'})

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-f6eb7b806a8ad95d/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")

model = AutoModelForSequenceClassification.from_pretrained("/content/bald")

training_args = TrainingArguments(
    output_dir=".",
    num_train_epochs=2,
    per_device_train_batch_size=48,
    per_device_eval_batch_size=48,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

trainer = BaalTransformersTrainer(
    model=model,
    args=training_args,
    tokenizer=None
)

In [11]:
def tokenize_function(examples):
    return tokenizer(examples['content'], examples['news_name_body'], max_length=200, truncation=True, padding="max_length")

In [12]:
tokenized_data = load_data.map(tokenize_function, batched=True)

Map:   0%|          | 0/307457 [00:00<?, ? examples/s]

In [13]:
output = trainer.predict(tokenized_data['train'])

In [14]:
predict_labels = np.argmax(output.predictions, axis=1)

In [15]:
data_for_predict['relevant_part'] = predict_labels
data_for_predict

<ipython-input-15-dcc4896a788a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_for_predict['relevant_part'] = predict_labels


,match_id,news_id,name,team1,team2,relevant_part,type,minute,content,news_name_body
21015,383449,140360494,Испания. Ла Лига,Барселона,Реал Мадрид,0,yellowcard,78,78 минута. Озил слишком долго подавал угловой.,Роналду установил рекорд результативности по г...
21016,383449,140360494,Испания. Ла Лига,Барселона,Реал Мадрид,0,yellowcard,78,"78 минута. А Гранеро схватил горчичник за то, ...",Роналду установил рекорд результативности по г...
21017,383449,140360494,Испания. Ла Лига,Барселона,Реал Мадрид,0,empty,80,"80 минута. ""Реал"" нейтрализует барселонский ко...",Роналду установил рекорд результативности по г...
21018,383449,140360494,Испания. Ла Лига,Барселона,Реал Мадрид,0,zamena,81,81 минута. Фабрегас меняет Тельо. Пеп исчерпал...,Роналду установил рекорд результативности по г...
21019,383449,140360494,Испания. Ла Лига,Барселона,Реал Мадрид,0,attention,83,83 минута. Четыре в два контратака Мадрида! Бе...,Роналду установил рекорд результативности по г...
...,...,...,...,...,...,...,...,...,...,...
328467,1013052,1027838659,Англия. Кубок,Манчестер Юнайтед,Арсенал,0,empty,95,95 минута. Сейчас пойдет компенсированное врем...,«МЮ» – «Арсенал». Окслейд-Чемберлен получил тр...
328468,1013052,1027838659,Англия. Кубок,Манчестер Юнайтед,Арсенал,0,attention,96,"96 минута. Все! 1:2 - впервые за 9 лет ""Арсена...",«МЮ» – «Арсенал». Окслейд-Чемберлен получил тр...
328469,1013052,1027838659,Англия. Кубок,Манчестер Юнайтед,Арсенал,0,empty,96,"96 минута. Руни снова забил ""канонирам"", но ег...",«МЮ» – «Арсенал». Окслейд-Чемберлен получил тр...
328470,1013052,1027838659,Англия. Кубок,Манчестер Юнайтед,Арсенал,0,empty,97,"97 минута. Спасибо, что были с нами! Завтра от...",«МЮ» – «Арсенал». Окслейд-Чемберлен получил тр...


In [16]:
result_data = pd.concat([train_data.loc[:21_015], data_for_predict, eval_data])
result_data

,match_id,news_id,name,team1,team2,relevant_part,type,minute,content,news_name_body
0,787156,158820706,Россия. Премьер-лига,Ростов,Амкар,0,empty,0,0 минута. Приветствуем Всех любителей самой по...,«Ростов» – «Амкар». Вавжиняк удален с поля & З...
1,787156,158820706,Россия. Премьер-лига,Ростов,Амкар,1,empty,0,0 минута. Предлагаем вашему вниманию прямую те...,«Ростов» – «Амкар». Вавжиняк удален с поля & З...
2,787156,158820706,Россия. Премьер-лига,Ростов,Амкар,0,empty,0,0 минута. Ростовчане весьма уверенно выступают...,«Ростов» – «Амкар». Вавжиняк удален с поля & З...
3,787156,158820706,Россия. Премьер-лига,Ростов,Амкар,0,empty,0,"0 минута. Более того, будь ""Ростов"" чуть удачл...",«Ростов» – «Амкар». Вавжиняк удален с поля & З...
4,787156,158820706,Россия. Премьер-лига,Ростов,Амкар,0,empty,0,"0 минута. ""Амкар"", подумать только, всего на 3...",«Ростов» – «Амкар». Вавжиняк удален с поля & З...
...,...,...,...,...,...,...,...,...,...,...
8247,787013,151814813,Россия. Премьер-лига,Рубин,ЦСКА,0,empty,90,90 минута. Вторая безголевая ничья в нынешнем ...,«Рубин» – ЦСКА. Карадениз получил травму & Пол...
8248,787013,151814813,Россия. Премьер-лига,Рубин,ЦСКА,0,empty,90,"90 минута. В целом, матч не был скучным, а во ...",«Рубин» – ЦСКА. Карадениз получил травму & Пол...
8249,787013,151814813,Россия. Премьер-лига,Рубин,ЦСКА,0,empty,90,90 минута. Ансальди совершает круг почета по с...,«Рубин» – ЦСКА. Карадениз получил травму & Пол...
8250,787013,151814813,Россия. Премьер-лига,Рубин,ЦСКА,0,empty,90,90 минута. Статистика матча. По владению мячом...,«Рубин» – ЦСКА. Карадениз получил травму & Пол...


In [19]:
result_data['relevant_part'].value_counts()

0    323580
1     13145
Name: relevant_part, dtype: int64

In [17]:
result_data.to_csv('summ_during_match.csv', index=False, encoding='utf-8')

In [18]:
!cp "/content/summ_during_match.csv" "/content/drive/MyDrive/RuSportSum/summ_during_match.csv"